In [1]:
import pandas  as pd
import numpy   as np
np.random.seed(42)
import seaborn as sns
from matplotlib            import pyplot          as plt
from IPython.display       import Image
from IPython.core.display  import HTML
from pylab                 import rcParams
from IPython.display       import Image
from scipy                 import stats           as ss
from sklearn               import model_selection as ms            
from sklearn               import preprocessing   as pp
from sklearn               import ensemble        as en
from sklearn               import neighbors       as nh
from sklearn               import linear_model    as lm
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics       import accuracy_score, f1_score,classification_report
from sklearn               import neighbors       as nh
from sklearn.ensemble      import RandomForestClassifier
from category_encoders.leave_one_out import LeaveOneOutEncoder as leave
from category_encoders.woe import WOEEncoder as woe
from category_encoders       import TargetEncoder
import xgboost as xgb
import sys
sys.path.append('/home/soturno/.pyenv/versions/3.10.4/envs/pa04/lib/python3.10/site-packages')
import scikitplot          as skplt
# pd.options.display.float_format = '{:,.2f}'.format
import warnings;   warnings.filterwarnings("ignore")

# sns.set_style("darkgrid")
# plt.style.use("default")
# rcParams['figure.figsize'] = 14,5

def precision_at_k(data,k=2000):   #2000 é apenas o default caso ngm passe o valor, poderia ser outro valor como 20000
    # reset index
    data=data.reset_index(drop=True) #necessário para q o index nao seja igual aos ids

    # create ranking order
    data['ranking']=data.index+1    #(para ñ começar do zero)

    data['precision_at_k']=data['response'].cumsum()/data['ranking']
    
    return data.loc[k,'precision_at_k']


def recall_at_k(data,k=2000):   #k poderia ser outro valor como 20000
    # reset index
    data=data.reset_index(drop=True) #necessário para q o index nao seja igual aos ids

    # create ranking order
    data['ranking']=data.index+1    #(para ñ começar do zero)

    data['recall_at_k']=data['response'].cumsum() /data['response'].sum()
    
    return data.loc[k,'recall_at_k']
def embed_features(learner, xs):
  """
  learner: fastai Learner used to train the neural net
  xs: DataFrame containing input variables. Categorical values are defined by their rank. 
 ::return:: copy of `xs` with embeddings replacing each categorical variable
  """
  xs = xs.copy()
  for i,col in enumerate(learn.dls.cat_names):
    
    # get matrix containing each row's embedding vector
    emb = learn.model.embeds[i]
    emb_data = emb(tensor(xs[col], dtype=torch.int64))
    emb_names = [f'{col}_{j}' for j in range(emb_data.shape[1])]
    
    # join the embedded category and drop the old feature column
    feat_df = pd.DataFrame(data=emb_data, index=xs.index,               
                           columns=emb_names)
    xs = xs.drop(col, axis=1)
    xs = xs.join(feat_df)
  return xs
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)
class TargetEncoderCV(TargetEncoder):
    """Cross-fold target encoder.
    """
    
    def __init__(self, n_splits=3, shuffle=True, cols=None):
        """Cross-fold target encoding for categorical features.
        
        Parameters
        ----------
        n_splits : int
            Number of cross-fold splits. Default = 3.
        shuffle : bool
            Whether to shuffle the data when splitting into folds.
        cols : list of str
            Columns to target encode.
        """
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.cols = cols
        

    def fit(self, X, y):
        """Fit cross-fold target encoder to X and y
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values.
            
        Returns
        -------
        self : encoder
            Returns self.
        """
        self._target_encoder = TargetEncoder(cols=self.cols)
        self._target_encoder.fit(X, y)
        return self

    
    def transform(self, X, y=None):
        """Perform the target encoding transformation.

        Uses cross-fold target encoding for the training fold,
        and uses normal target encoding for the test fold.

        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode

        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """

        # Use target encoding from fit() if this is test data
        if y is None:
            return self._target_encoder.transform(X)

        # Compute means for each fold
        self._train_ix = []
        self._test_ix = []
        self._fit_tes = []
        kf = KFold(n_splits=self.n_splits, shuffle=self.shuffle)
        for train_ix, test_ix in kf.split(X):
            self._train_ix.append(train_ix)
            self._test_ix.append(test_ix)
            te = TargetEncoder(cols=self.cols)
            if isinstance(X, pd.DataFrame):
                self._fit_tes.append(te.fit(X.iloc[train_ix,:],
                                            y.iloc[train_ix]))
            elif isinstance(X, np.ndarray):
                self._fit_tes.append(te.fit(X[train_ix,:],
                                            y[train_ix]))
            else:
                raise TypeError('X must be DataFrame or ndarray')

        # Apply means across folds
        Xo = X.copy()
        for ix in range(len(self._test_ix)):
            test_ix = self._test_ix[ix]
            if isinstance(X, pd.DataFrame):
                Xo.iloc[test_ix,:] = \
                    self._fit_tes[ix].transform(X.iloc[test_ix,:])
            elif isinstance(X, np.ndarray):
                Xo[test_ix,:] = \
                    self._fit_tes[ix].transform(X[test_ix,:])
            else:
                raise TypeError('X must be DataFrame or ndarray')
        return Xo

            
    def fit_transform(self, X, y=None):
        """Fit and transform the data via target encoding.
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values (required!).

        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """
        return self.fit(X, y).transform(X, y)
    
sns.set_style("whitegrid")
plt.style.use("classic")
rcParams['figure.figsize'] = 14,5

In [25]:
df5=pd.read_csv('df5',low_memory=False)

# rescaling


# normalization
mms_age=pp.MinMaxScaler()
df5['age']=mms_age.fit_transform(df5[['age']].values)

mms_vintage=pp.MinMaxScaler()
df5['vintage']=mms_vintage.fit_transform(df5[['vintage']].values)

# mms_annual_premium=pp.MinMaxScaler()

# robust scaler
rs_annual_premium= pp.RobustScaler()
df5['annual_premium'] = rs_annual_premium.fit_transform(df5[['annual_premium']].values)

# standardization
# ss_annual_premium=pp.StandardScaler()


# ONE HOT
df5=pd.get_dummies(df5,prefix='vehicle_age',columns=['vehicle_age'])
df5=pd.get_dummies(df5,prefix='gender',columns=['gender'])


# TARGET
target_encode_region_code=df5.groupby('region_code')['response'].mean()
df5.loc[:,'region_code']=df5['region_code'].map(target_encode_region_code)

# FREQUENCY
fe_policy_sales_channel=df5.groupby('policy_sales_channel').size()/len(df5)
df5.loc[:,'policy_sales_channel']=df5['policy_sales_channel'].map(fe_policy_sales_channel)


X=df5.drop('response',axis=1)
y=df5['response'].copy()

x_train, x_validation, y_train, y_validation = ms.train_test_split(X,y,test_size=0.20,stratify=y,random_state=42)

df5=pd.concat ([x_train, y_train], axis=1) 
#################################################################################################
# rescaling

# normalization
# age
# x_validation.loc[:,'age']=mms_age.transform(x_validation[['age']].values)
# # vintage
# x_validation.loc[:,'vintage']=mms_vintage.transform(x_validation[['vintage']].values)


# # robust scaler
# # annual_premium
# x_validation.loc[:,'annual_premium']=rs_annual_premium.transform(x_validation[['annual_premium']].values)


# # standardization
# # x_validation.loc[:,'annual_premium']=ss_annual_premium.transform(x_validation[['annual_premium']].values)

# # ONE HOT
# # gender
# x_validation=pd.get_dummies(x_validation,prefix='gender',columns=['gender'])
# # vehicle_age
# x_validation=pd.get_dummies(x_validation,prefix='vehicle_age',columns=['vehicle_age'])


# # TARGET
# # region_code
# x_validation.loc[:,'region_code']=x_validation.loc[:,'region_code'].map(target_encode_region_code) 
# # x_validation.loc[:,'gender']=x_validation.loc[:,'gender'].map(target_encode_gender)


# # FREQUENCY
# # policy_sales_channel
# x_validation.loc[:,'policy_sales_channel']=x_validation['policy_sales_channel'].map(fe_policy_sales_channel)


# fillna
# x_validation=x_validation.fillna(0) # tem regiões (region_code) no teste q n estão no treino

In [23]:
# feature selection
# # model definition
# forest=en.ExtraTreesClassifier(n_estimators=250,random_state=0,n_jobs=-1) 

# # recebe os dados de treino e a variável resposta
# # data preparation
# x_train_n=df5.drop(['id','response'],axis=1)
# y_train_n=y_train.values
# forest.fit(x_train_n,y_train_n)

# importances=forest.feature_importances_
# std=np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
# indices=np.argsort(importances)[::-1]  # índice é a importância das árvores ornada pelo maior valor

# # print the feature ranking
# print('Feature ranking')
# df=pd.DataFrame()
# for i, j in zip(x_train_n,forest.feature_importances_): # zip é pra arrumar as colunas com os valores
#     aux=pd.DataFrame({'feature':i,'importance':j},index=[0])
#     df=pd.concat([df,aux],axis=0)
    
# print(df.sort_values('importance',ascending=False))



# df1=df.sort_values(by='importance',ascending=False)
# df1=df1.drop('importance',axis=1)
# df1['feature'].tolist()

In [26]:
# cols_selected=df1['feature'].tolist()
cols_selected=['vintage',
 'annual_premium',
 'age',
 'region_code',
 'vehicle_damage',
 'policy_sales_channel',
 'previously_insured',
 'vehicle_age_below_1_year',
 'vehicle_age_between_1_2_years',
 'vehicle_age_over_2_years',
 'gender_0',
 'gender_1',
 'driving_license']

# cols_selected=['previously_insured', 'annual_premium', 'vintage', 'age',
#        'region_code', 'policy_sales_channel', 'driving_license',
#        'vehicle_damage','gender_Female', 'gender_Male',
#        'vehicle_age_below_1_year', 'vehicle_age_between_1_2_years',
#        'vehicle_age_over_2_years']

x_train=df5[cols_selected]
x_val=x_validation[cols_selected]
y_val=y_validation.copy()

In [ ]:
# # accumulative gain
# skplt.metrics.plot_cumulative_gain(y_val,yhat_knn);

# df8=x_validation.copy()
# df8['response']=y_validation.copy()       #response = propensity score

# # propensity score
# df8['score']=yhat_knn[:,1].tolist()

# # sorte clients by propensity score
# df8=df8.sort_values('score',ascending=False)

# # reset index
# df8=df8.reset_index(drop=True)

# # create ranking order
# df8['ranking']=df8.index+1    #(para ñ começar do zero)

# df8['precision_at_k']=df8['response'].cumsum()/df8['ranking']

# df8[['id','response','score','ranking','precision_at_k']].head(10)

# precision_at_50=precision_at_k(df8,k=50)
# print(precision_at_50)


# recall_at_50=recall_at_k(df8,k=50)
# print(recall_at_50)

In [33]:
# model definition
knn_model=nh.KNeighborsClassifier(n_neighbors=7)     

# model training
knn_model.fit(x_train,y_train)      

# # model prediction                 
# yhat_knn=knn_model.predict_proba(x_val)
yhat_knn=knn_model.predict(x_val)

print("Accuracy: %.2f%%" % (accuracy_score(y_val, yhat_knn)*100))

Accuracy: 86.03%


In [31]:
# model definition
lr_model=lm.LogisticRegression(random_state=42)

# model training
lr_model.fit(x_train,y_train)

# model prediction
# yhat_lr=lr_model.predict_proba(x_val);
yhat_lr=lr_model.predict(x_val);

print("Accuracy: %.2f%%" % (accuracy_score(y_val, yhat_lr)*100))

Accuracy: 87.74%


In [ ]:
# # model definition
# et=en.ExtraTreesClassifier(n_estimators=1000,n_jobs=-1,random_state=42)    # njobs-1 é pra usar os cores da minha máquina

# # model training
# et.fit(x_train,y_train)

# # model predict
# yhat_et=et.predict_proba(x_val)

# # accumulative gain
# skplt.metrics.plot_cumulative_gain(y_val,yhat_et);

# df8=x_validation.copy()
# df8['response']=y_validation.copy()       #response = propensity score

# # propensity score
# df8['score']=yhat_et[:,1].tolist()

# # sorte clients by propensity score
# df8=df8.sort_values('score',ascending=False)

# # reset index
# df8=df8.reset_index(drop=True)

# # create ranking order
# df8['ranking']=df8.index+1    #(para ñ começar do zero)

# df8['precision_at_k']=df8['response'].cumsum()/df8['ranking']

# df8[['id','response','score','ranking','precision_at_k']].head(10)

# precision_at_50=precision_at_k(df8,k=50)
# print(precision_at_50)


# recall_at_50=recall_at_k(df8,k=50)
# print(recall_at_50)

In [34]:
xgb_model = xgb.XGBClassifier(random_state=42)

# model training
xgb_model.fit(x_train,y_train)

# model prediction
# yhat_xgb=xgb_model.predict_proba(x_val);
yhat_xgb=xgb_model.predict(x_val);

print("Accuracy: %.2f%%" % (accuracy_score(y_val, yhat_xgb)*100))

Accuracy: 87.73%
